In [95]:
import openai
from dotenv import find_dotenv, load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi
import streamlit as st
from langchain import OpenAI
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import os

current_directory = os.getcwd()
load_dotenv(current_directory + '/conf.env')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [96]:
# Download sample video transcript
transcript = YouTubeTranscriptApi.get_transcript('CR8Niz9DrWU&t=6s')
res = ''
for txt in transcript:
    res += ' ' + txt['text']

with open(current_directory + '/interview_1.txt', 'w', encoding='utf-8') as interview:
    interview.write(res)  

In [97]:
def generate_response(txt):
    # Instantiate the LLM model
    llm = OpenAI(temperature = 0.0, 
                 openai_api_key = OPENAI_API_KEY)
    
    # Split text
    text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 4000,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True,)
    
    texts = text_splitter.split_text(txt)
    # Create multiple documents
    docs = [Document(page_content=t) for t in texts]
    # Text summarization
    chain = load_summarize_chain(llm, chain_type='map_reduce')
    return chain.run(docs)

summerized_text = generate_response(res)

In [116]:
from langchain.prompts import PromptTemplate

INTERVIEW_QUESTION_PROMPT = """ We asked a candidate an interview question of {interview_question}. 
Here is the sumerized response: {candidate_response} 

Question : How do you evaluate the candidate's response?

Instructions:

1) The story should be specific about an industrial project in the domain of 
   engineering product or marketing.
2) The story should be clear and easy to understand.
3) The story should be concise and to the point.
4) The story should be relevant to the question.

Please use the following template for your response :

Your general ranking should be only one of these choices:
1) weak, 
2) average, 
3) strong

With the Explanation of your ranking:"""

prompt_template = PromptTemplate(
    input_variables =["candidate_response", "interview_question"],
    template = INTERVIEW_QUESTION_PROMPT
)

query = prompt_template.format(
        candidate_response = summerized_text,
        interview_question = "Tell me about the time that you solved a customer's problem"
    )

In [117]:
llm = OpenAI(temperature = 0.0, openai_api_key = OPENAI_API_KEY)
print(llm(query))


The candidate's response was strong. They provided a detailed story about a successful project they worked on that was relevant to the question. They also provided tips for Amazon product interviews, which showed their knowledge and experience in the field.


In [118]:
CUSTOMER_OBSESSION ="""It emphasizes the importance of putting the customer at the center of everything Amazon does. 
Here's a more detailed explanation of this principle: 

Customer-Centric Approach: This means that every decision, product, service, and process is designed with the customer's needs and 
preferences in mind. Amazon strives to understand its customers deeply, to anticipate their needs, and to exceed their expectations.

Long-Term Thinking: Customer Obsession is not just about satisfying customers in the short term. It's about building long-term 
relationships and trust. It can create lasting loyalty and generate sustainable business growth.

Customer Feedback: Actively seeks and listens to customer feedback. Customer reviews, ratings, and comments are not only encouraged but 
are also used to make improvements and identify areas where the company can do better. Customers' voices are considered valuable 
sources of insights.

Data-Driven Decision-Making: It relies heavily on data and analytics to understand customer behavior and preferences. This data-driven 
approach helps the company make informed decisions and continuously improve its offerings to better serve customers. Company should use 
metrics and feedback to measure its success in delivering exceptional customer experiences.
"""

OWNERSHIP = """ it encourages employees to act as if they are the owners of the company and take full responsibility for their work 
and decisions. This means taking the initiative, being proactive, and seeking out opportunities to make a difference. 
It also means taking pride in one's work and striving for excellence.Here's a more detailed explanation of this principle:

End-to-End Responsibility: Ownership means taking complete ownership of a project or task from start to finish. 
This includes not only the execution but also the planning, coordination, and follow-through. Employees are expected to 
see their projects through to completion and ensure they meet their goals and deliver value. This means that ownership 
includes considering the long-term impact and consequences of decisions, not just short-term gains.

Deliver Results: Ownership involves setting high standards for performance and delivering results. It has a strong emphasis on 
measurable outcomes, and employees are held accountable for achieving their goals. Ownership means not just working hard but also 
working smart to achieve meaningful results.
"""

RANDOM_STORY = """Once upon a time, in a small, quiet town nestled between rolling hills and a shimmering lake, there 
stood a quaint little bookstore called "The Enchanted Tome." Its owner, an elderly woman named Eliza, had a secret known to only a few.

The bookstore was a magical place, and its shelves were filled with books that came to life. When a customer opened a book 
and began to read, they were transported into the world of the story. The characters would come alive, and readers could 
interact with them as if they were old friends.

One sunny morning, a curious young girl named Lily entered the bookstore. She had heard whispers of its enchantment and was 
eager to see if the rumors were true. With a sense of excitement, she approached the dusty shelves and chose a thick, 
leather-bound volume titled "The Adventures of Captain Orion."

As Lily opened the book, a gust of wind swept her off her feet and into a world of pirate ships and treasure hunts. 
She found herself on the deck of the pirate ship, face to face with the daring Captain Orion himself. With a mischievous 
grin, he invited her to join the crew and search for buried treasure on a remote island.

Lily's heart raced as she navigated through dangerous waters, encountered mermaids, and outwitted rival pirates. The 
adventure was more thrilling than anything she had ever imagined. And all the while, the pages of the book seemed to 
come alive with vivid illustrations that leaped from the page.

Days turned into weeks as Lily lived out her pirate dreams. But eventually, it was time to return to the real world. 
Captain Orion and his crew waved her goodbye, promising that their adventures would live on in the pages of the book.

Back in the bookstore, Lily closed the book with a satisfied sigh. She turned to Eliza, who had been watching with a 
knowing smile.

"It's real," Lily exclaimed.

Eliza nodded. "Every book here holds a world of its own. And every reader who dares to open them becomes a part of that world, 
if only for a little while."

From that day forward, Lily became a frequent visitor to "The Enchanted Tome," and she discovered countless new adventures 
within its pages. And so did others who entered the magical bookstore, each finding their own unique journeys in the stories 
that came to life.

"The Enchanted Tome" remained a beloved secret in the quiet town, where anyone could escape reality for a while and explore 
the limitless worlds hidden within the pages of a book.

And as long as Eliza tended to her store, the magic of reading would continue to thrive in that enchanting corner of the world.

The End."""

In [119]:
random_query = prompt_template.format(
        candidate_response = RANDOM_STORY,
        interview_question = "Tell me about the time that you solved a customer's problem"
    )

llm = OpenAI(temperature = 0.0, openai_api_key = OPENAI_API_KEY)
print(llm(random_query))


The candidate's response was [weak/average/strong] because [provide explanation]. The story was not specific to an industrial project in the domain of engineering product or marketing, and it was not relevant to the question. The story was also not concise and to the point, making it difficult to understand.


In [120]:
from langchain.embeddings import OpenAIEmbeddings
import numpy as np
#Define embedding
embedding = OpenAIEmbeddings(client=None)

CUSTOMER_OBSESSION_VECTOR = embedding.embed_query(CUSTOMER_OBSESSION)
OWNERSHIP_VECTOR = embedding.embed_query(OWNERSHIP)
SUMMERIZED_TEXT_VECTOR = embedding.embed_query(summerized_text)
RANDOM_STORY_VECTOR = embedding.embed_query(RANDOM_STORY)


def inner_vector_product(vector_1, vector_2):
    """
    This function calculates the inner vector product of two vectors.
    """
    import numpy as np
    a = np.array(vector_1)
    b = np.array(vector_2)

    return np.dot(a, b)



print(inner_vector_product(SUMMERIZED_TEXT_VECTOR, CUSTOMER_OBSESSION_VECTOR))
print(inner_vector_product(SUMMERIZED_TEXT_VECTOR, RANDOM_STORY_VECTOR))
print(inner_vector_product(SUMMERIZED_TEXT_VECTOR, OWNERSHIP_VECTOR))




0.776041445156155
0.7260945145810024
0.7332620459010506
